<a href="https://colab.research.google.com/github/ChrisFJardine/ChrisFJardine/blob/main/3CRC%20Results%20Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Map Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import requests
import time
from datetime import datetime
from bs4 import BeautifulSoup
from decimal import Decimal
import pandas as pd

In [2]:
def getSoup(page):
  soup = BeautifulSoup(page.text, 'html5lib')
  return (soup)


In [5]:
def getRunDate(soup):
  dateString = soup.find('h4').text
  runDate = datetime.strptime(dateString, '%d/%m/%Y').date()
  return(runDate)

In [6]:
def getCourse(soup):
  if soup.find('h2')is None:
    print('error in getCourse')
    course = 'getCourse Error'
  else:
      course = soup.find('h2').text
  return(course)

In [7]:
def getRunDetails(soup):
  columns = ['Athlete Place','Athlete Name', 'Athlete Gender','Run Time','Avg Pace','Points', 'Run ClassID','Run Length','Run Distance','Run Location','Run Date']
  runDetails = pd.DataFrame(columns = columns)
  course = getCourse(soup).replace('Results','')
  runDate = getRunDate(soup)
  for record in soup.find('table', attrs={'class':'table'}).find_all(['tr']):
    if record.find('img') is not None:    
      runClassID = record.find('img')['class'][0]
      headerString = record.get_text().split()
      if 'Run' in headerString:
        runType = (headerString[headerString.index('Run') -1])
        for word in headerString:
          if 'km' in word:
            dist  = Decimal(word.replace('km','').replace(')','')) 
    row = [td.text for td in record.find_all(['td'])]
    row.append(runClassID)
    row.append(runType)
    row.append(dist)
    row.append(course)
    row.append(runDate)
    if len(row) == 11:
      runDetails.loc[len(runDetails)] = row
  return(runDetails) 


In [14]:
def get_pages(urlbase, startPageNo, pagesToCheck):
  columns = ['Athlete Place','Athlete Name', 'Athlete Gender','Run Time','Avg Pace','Points', 'Run ClassID','Run Length','Run Distance','Run Location','Run Date']
  allResults = pd.DataFrame(columns = columns)
  for i in range (startPageNo, startPageNo + pagesToCheck):          #150 is start of 2022 season. Checks for 50 pages
    url = urlbase + str(i)
    try:
      page = requests.get(url)
    except:
      print('Exception ' + str(page.status_code))  
    if page.status_code == 200:
        soup = getSoup(page)
        runDetails = getRunDetails(soup)
        allResults = pd.concat([allResults, runDetails])
        time.sleep(1)
  return(allResults)

In [41]:
urlbase = "https://theautomatedclub.com/3CRC/Result/Details/"
results = get_pages(urlbase, 150, 20)
results = results.drop_duplicates()

In [ ]:
display(results[results['Athlete Name'].str.contains("Chris Jardine")])